# Utilities

> Utility functions used in `pybx` to calculate anchor boxes, among others.

In [ ]:
#| default_exp utils

In [ ]:
#| export
import inspect

import numpy as np

from pybx.ops import __ops__, get_op
from pybx.basics import get_bx

from fastcore.foundation import L

In [ ]:
# | export
def get_edges(image_sz: tuple, feature_sz: tuple, op="noop"):
    """Generate offsetted top `(x_min, y_min)` or bottom edges `(x_max, y_max)`
    coordinates of a given feature size based on op.
    if `op` is `noop`, gets the top edges.
    if `op` is `add`, gets the bottom edges.
    :param op: operation for calculating edges, either 'add' 'sub' 'noop'
    :param image_sz: tuple of `(W, H)` of an image
    :param feature_sz: tuple of `(W, H)` of a channel
    :return: offsetted edges of each feature
    """
    assert (
        len(image_sz) == 2
    ), f"{inspect.stack()[0][3]} of {__name__}: Expected image_sz of len 2, got {len(image_sz)}"

    assert (
        op in __ops__
    ), f"{inspect.stack()[0][3]} of {__name__}: Operator not in allowed operations: {__ops__}"
    w, h = image_sz
    nx, ny = feature_sz
    diag_edge_ofs = w / nx, h / ny
    op_ = get_op(op)
    x_ = op_(np.linspace(0, w, nx + 1), diag_edge_ofs[0])
    y_ = op_(np.linspace(0, h, ny + 1), diag_edge_ofs[1])
    mesh = np.meshgrid(x_, y_)
    edges = np.stack([m.flatten() for m in mesh], axis=-1)
    return edges


def validate_boxes(coords, image_sz, feature_sz, clip=True, min_visibility=0.25):
    """Validate calculated anchor box coords.
    :param coords: anchor box coordinates
    :param image_sz: tuple of (width, height) of an image
    :param feature_sz: tuple of (width, height) of a channel
    :param clip: whether to apply np.clip
    :param min_visibility: minimum visibility dictates the condition for a box to be considered valid. The value corresponds to the
    ratio of expected area to the calculated area after clipping to image dimensions.
    :return: anchor box coordinates in [pascal_voc] format
    """
    _max = max(image_sz[0], image_sz[1])
    # clip the boxes to image dimensions
    bxs = get_bx(coords.clip(0, _max)) if clip else get_bx(coords)
    # check if the area of the bounding box is fitting the minimum area criterion
    min_area = (
        (image_sz[0] / feature_sz[0]) * (image_sz[1] / feature_sz[1]) * min_visibility
    )
    bxs = L(list(b._coords) for b in bxs if b.area > min_area)
    return bxs


In [ ]:
np.random.seed(42)
coords = np.random.random((100, 4))
coords[:2]

array([[0.37454012, 0.95071431, 0.73199394, 0.59865848],
       [0.15601864, 0.15599452, 0.05808361, 0.86617615]])

In [ ]:
get_bx(coords)[0]._coords

array([0.37454012, 0.95071431, 0.73199394, 0.59865848])

In [ ]:
validate_boxes(coords, (1, 1), (1, 1), clip=True, min_visibility=0.25)

(#14) [[0.07455064367977082, 0.9868869366005173, 0.7722447692966574, 0.1987156815341724],[0.8631034258755935, 0.6232981268275579, 0.3308980248526492, 0.06355835028602363],[0.907566473926093, 0.24929222914887494, 0.41038292303562973, 0.7555511385430487],[0.8074401551640625, 0.8960912999234932, 0.3180034749718639, 0.11005192452767676],[0.22793516254194168, 0.4271077886262563, 0.8180147659224931, 0.8607305832563434],[0.040775141554763916, 0.5908929431882418, 0.6775643618422824, 0.016587828927856152],[0.3410663510502585, 0.11347352124058907, 0.9246936182785628, 0.877339353380981],[0.5296505783560065, 0.24185229090045168, 0.09310276780589921, 0.8972157579533268],[0.6420316461542878, 0.08413996499504883, 0.16162871409461377, 0.8985541885270792],[0.6064290596595899, 0.009197051616629648, 0.1014715428660321, 0.6635017691080558]...]

In [ ]:
#| export
def as_tuple(x):
    """Get x as a tuple (x, x) if not already a tuple.

    Parameters
    ----------
    x : (int, tuple)
        Item that needs to be converted to a tuple.
    """    
    return (x, x) if isinstance(x, int) else x

In [ ]:
as_tuple(2)

(2, 2)